In [1]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/home/marco/anaconda3/bin/python
3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=3, releaselevel='final', serial=0)


In [2]:
sc.stop()

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession\
    .builder\
    .appName("Feature Enginering")\
    .config("spark.driver.memory", "15g")\
    .getOrCreate()

sc = spark.sparkContext

In [3]:
sc._conf.get('spark.driver.memory')

'15g'

In [4]:
sc._conf.getAll()

[('spark.driver.memory', '15g'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.host', '192.168.3.5'),
 ('spark.app.id', 'local-1638338500846'),
 ('spark.app.name', 'Feature Enginering'),
 ('spark.app.startTime', '1638338500776'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '46755')]

In [5]:
sc.version

'3.0.3'

![imagenes/](img/DataPreparation.png)

#### Carga de fuentes de información

Cabeceras:
    - origen: numero de origen,
    - destino: numero de destino,
    - tipo: tipo de llamada,
    - segundos: duración de llamada (seg),
    - llamadas: numero de llmadas, 
    - mensajes: numero de sms

In [6]:
from pyspark.sql.types import StructField, StructType, StringType,DoubleType
custom_schema = StructType([
    StructField("origen", StringType(), True),
    StructField("destino", StringType(), True),
    StructField("tipo", StringType(), True),
    StructField("segundos", DoubleType(), True),
    StructField("llamadas", DoubleType(), True),
    StructField("mensajes", DoubleType(), True)
])

mtDf = spark.read.format("csv")\
        .schema(custom_schema)\
        .option("header","false")\
        .load("../rawData/mt/week2/20150310.csv.sanitaized")

mtDf.show(truncate=True)

mtDf.filter(mtDf.tipo!="MOBILE").show()

In [7]:
moDf = spark.read.format("csv")\
        .schema(custom_schema)\
        .option("header","false")\
        .load("../rawData/mo/week2/20150310.csv")

In [8]:
moDf.show()

+--------------------+--------------------+-------+--------+--------+--------+
|              origen|             destino|   tipo|segundos|llamadas|mensajes|
+--------------------+--------------------+-------+--------+--------+--------+
|dad878015fcbecfda...|261d09ee269bd79c0...| MOBILE|    23.0|     2.0|     0.0|
|3b3af9f22ec61f89c...|f228da1389ba37cc1...| MOBILE|    57.0|     1.0|     0.0|
|840addc6fb560bd0a...|54417a8d60abcafb8...| MOBILE|    52.0|     1.0|     0.0|
|696e2cd1f2874f0ec...|02782fd0428887eda...| MOBILE|    35.0|     2.0|     0.0|
|e5f9dfeb9b97f392e...|7ed5dc16f1b74c11d...| MOBILE|   379.0|     1.0|     0.0|
|ecaf1439ea61a47c6...|65f09eaaafd0b7275...| MOBILE|   228.0|     2.0|     0.0|
|cbb5f445dff5ec7ca...|84a64468e4b26872b...| MOBILE|    42.0|     2.0|     0.0|
|5b61a1d5d08bfbccf...|a1ced1b13747c0cc9...| MOBILE|    43.0|     2.0|     0.0|
|178f76bc7bc5ad28e...|484342a23fc025e82...| MOBILE|   141.0|     2.0|     0.0|
|243a442812e365849...|12b104daa5924ec49...| MOBILE| 

In [9]:
#moDf.filter(moDf.tipo!="MOBILE").count()   # 994,258

In [10]:
#moDf.filter(moDf.tipo=="MOBILE").count()   # 7,151,552

In [11]:
from pyspark.sql import functions as F

In [12]:
#moDf.groupBy("origen").agg(F.sum("segundos").alias("sum_segundos"), \
 #                          F.countDistinct("destino").alias("destinos"), \
  #                         F.sum("llamadas").alias("sum_llamadas"), \
   #                        F.sum("mensajes").alias("sum_sms")).show()

In [13]:
moDf.groupBy("tipo").agg(F.count("*"),F.countDistinct(moDf.origen)).show()

+-------------+--------+-------------+
|         tipo|count(1)|count(origen)|
+-------------+--------+-------------+
|     LANDLINE|  189466|       162658|
|       OTHERS|    2720|         2466|
|      SERVICE|  714568|       632290|
|       MOBILE| 7151552|      1986899|
|INTERNATIONAL|   87504|        64002|
+-------------+--------+-------------+



In [14]:
#moDf.groupBy("origen","tipo").agg(F.when(F.col("tipo")=="MOBILE",F.sum("segundos")).otherwise(0).alias("sum_segundos")\
#                          ).show()

In [15]:
moDfInter01=moDf\
    .withColumn("segMobile", F.when(moDf.tipo == "MOBILE", moDf.segundos).otherwise(0))\
    .withColumn("segService", F.when(moDf.tipo == "SERVICE", moDf.segundos).otherwise(0))\
    .withColumn("segInternational", F.when(moDf.tipo == "INTERNATIONAL", moDf.segundos).otherwise(0))\
    .withColumn("segOthers", F.when(~((moDf.tipo=="INTERNATIONAL")|\
                                            (moDf.tipo=="SERVICE")|\
                                            (moDf.tipo=="MOBILE")), moDf.segundos).otherwise(0))\
    .withColumn("llaMobile", F.when(moDf.tipo == "MOBILE", moDf.llamadas).otherwise(0))\
    .withColumn("llaService", F.when(moDf.tipo == "SERVICE", moDf.llamadas).otherwise(0))\
    .withColumn("llaInternational", F.when(moDf.tipo == "INTERNATIONAL", moDf.llamadas).otherwise(0))\
    .withColumn("llaOthers", F.when(~((moDf.tipo=="INTERNATIONAL")|\
                                            (moDf.tipo=="SERVICE")|\
                                            (moDf.tipo=="MOBILE")), moDf.llamadas).otherwise(0))\
    .withColumn("desMobile", F.when(moDf.tipo == "MOBILE", moDf.destino))\
    .withColumn("desService", F.when(moDf.tipo == "SERVICE", moDf.destino))\
    .withColumn("desInternational", F.when(moDf.tipo == "INTERNATIONAL", moDf.destino))\
    .withColumn("desOthers", F.when(~((moDf.tipo=="INTERNATIONAL")|\
                                            (moDf.tipo=="SERVICE")|\
                                            (moDf.tipo=="MOBILE")), moDf.destino))

In [16]:
#moDfInter01.count()

In [17]:
moDfInter01 = moDfInter01.groupBy("origen")\
            .agg(\
                 ### Agregando en Segundos
                 F.sum("segundos").alias("sumSeg")\
                ,F.sum("segMobile").alias("sumSegMobile")\
                ,F.sum("segService").alias("sumSegService")\
                ,F.sum("segInternational").alias("sumSegInternational")\
                ,F.sum("segOthers").alias("sumSegOthers")\
                 ### Agregando en llamadas
                ,F.sum("llamadas").alias("sumLla")\
                ,F.sum("llaMobile").alias("sumLlaMobile")\
                ,F.sum("llaService").alias("sumLlaService")\
                ,F.sum("llaInternational").alias("sumLlaInternational")\
                ,F.sum("llaOthers").alias("sumLlaOthers")\
                 ### Agregando destinos distintos
                ,F.countDistinct("destino").alias("nroDest")\
                ,F.countDistinct("desMobile").alias("nroDestMobile")\
                ,F.countDistinct("desService").alias("nroDestService")\
                ,F.countDistinct("desInternational").alias("nroDestInternational")\
                ,F.countDistinct("desOthers").alias("nroDestOthers")\
                )

In [ ]:
moDfInter01.count()

Py4JJavaError: An error occurred while calling o257.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 16.0 failed 1 times, most recent failure: Lost task 2.0 in stage 16.0 (TID 415, 192.168.3.5, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:57)
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:335)
	at org.apache.spark.io.ReadAheadInputStream.<init>(ReadAheadInputStream.java:105)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillReader.<init>(UnsafeSorterSpillReader.java:74)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillWriter.getReader(UnsafeSorterSpillWriter.java:159)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.getSortedIterator(UnsafeExternalSorter.java:523)
	at org.apache.spark.sql.execution.UnsafeKVExternalSorter.sortedIterator(UnsafeKVExternalSorter.java:206)
	at org.apache.spark.sql.execution.aggregate.HashA

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/marco/Downloads/spark-3.0.3-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/marco/Downloads/spark-3.0.3-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/home/marco/Downloads/spark-3.0.3-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving


In [19]:
moDfInter01.limit(10).toPandas().head()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:40597)
Traceback (most recent call last):
  File "/home/marco/Downloads/spark-3.0.3-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/marco/Downloads/spark-3.0.3-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:40597)

In [ ]:
moDf.filter(moDf.origen=="5f96f009c808466497ce0a5e19126cd9144be71c3245766948a57a2e").show()

In [ ]:
### Agregando Proporciones
### Agregando proporciones en segundos con un artificio para evitar la división por 0
### Agregando proporciones en llamadas con un artificio para evitar la división por 0
### Agregando proporciones en nro de destinos con un artificio para evitar la división por 0
moDfInter01 = moDfInter01\
    .withColumn("PropSegMobile", F.round((moDfInter02.sumSegMobile+0.5)/(moDfInter02.sumSeg+1),5))\
    .withColumn("PropSegService", F.round((moDfInter02.sumSegService+0.5)/(moDfInter02.sumSeg+1),5))\
    .withColumn("PropSegInternational", F.round((moDfInter02.sumSegInternational+0.5)/(moDfInter02.sumSeg+1),5))\
    .withColumn("PropLlaMobile", F.round((moDfInter02.sumLlaMobile+0.5)/(moDfInter02.sumLla+1),5))\
    .withColumn("PropLlaService", F.round((moDfInter02.sumLlaService+0.5)/(moDfInter02.sumLla+1),5))\
    .withColumn("PropLlaInternational", F.round((moDfInter02.sumLlaInternational+0.5)/(moDfInter02.sumLla+1),5))\
    .withColumn("PropNroDestMobile", F.round((moDfInter02.nroDestMobile+0.5)/(moDfInter02.nroDest+1),5))\
    .withColumn("PropNroDestService", F.round((moDfInter02.nroDestService+0.5)/(moDfInter02.nroDest+1),5))\
    .withColumn("PropNroDestInternational", F.round((moDfInter02.nroDestInternational+0.5)/(moDfInter02.nroDest+1),5))

In [ ]:
moDfInter01.limit(10).toPandas().head()

In [ ]:
moDfInter01.count()

In [ ]:
sc